In [1]:
import pandas as pd

In [2]:
X_train = pd.read_csv("/content/drive/MyDrive/nlp/training_content_all.csv").dropna()
X_train = X_train[X_train["form_type"] != "4"]
print(len(X_train))
y_train = X_train["label"]
X_train.head(3)

122429


,file_number,ticker,form_type,label,content
10,10,UAL,8-K,1,earliest event registrant specified principal ...
12,12,NRG,SC 13G/A,0,true false false false false false option gran...
13,13,MRO,SC 13G/A,0,true false false false false false option gran...


In [3]:
from keras.preprocessing import text, sequence
from keras.preprocessing.sequence import pad_sequences


# use keras to create a Tokenizer object
tokenizer = text.Tokenizer(num_words=10000)  # limit to the num_words most important ones
tokenizer.fit_on_texts(list(X_train['content']))
tokenized_texts = tokenizer.texts_to_sequences(X_train['content'])
X = sequence.pad_sequences(tokenized_texts, maxlen=500)

In [ ]:
embedding_matrix = np.zeros((len(tokenizer.index_word)+1, 300))
for word, i in tokenizer.index_word:
  if i > len(tokenizer.index_word):
    continue
  embedding_vector = 

In [4]:
from keras.models import Sequential
from keras.layers import Input, Dense, LSTM, Embedding

model = Sequential()

embedding_size = 32
model.add(Embedding(len(tokenizer.index_word)+1, embedding_size))
model.add(LSTM(32, dropout=0, recurrent_dropout=0))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [5]:
model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          1172064   
                                                                 
 lstm (LSTM)                 (None, 32)                8320      
                                                                 
 dense (Dense)               (None, 32)                1056      
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 1,181,473
Trainable params: 1,181,473
Non-trainable params: 0
_________________________________________________________________


In [6]:
model.fit(X, X_train['label'], epochs=10, batch_size=32, validation_split=0.1)

Epoch 1/10
3444/3444 [==============================] - 80s 22ms/step - loss: 0.6471 - accuracy: 0.5628 - val_loss: 0.7007 - val_accuracy: 0.4945
Epoch 2/10
3444/3444 [==============================] - 74s 22ms/step - loss: 0.6276 - accuracy: 0.5928 - val_loss: 0.7096 - val_accuracy: 0.5263
Epoch 3/10
3444/3444 [==============================] - 75s 22ms/step - loss: 0.6178 - accuracy: 0.6064 - val_loss: 0.7286 - val_accuracy: 0.5238
Epoch 4/10
3444/3444 [==============================] - 75s 22ms/step - loss: 0.6025 - accuracy: 0.6209 - val_loss: 0.7647 - val_accuracy: 0.5291
Epoch 5/10
3444/3444 [==============================] - 75s 22ms/step - loss: 0.5914 - accuracy: 0.6343 - val_loss: 0.7720 - val_accuracy: 0.5247
Epoch 6/10
3444/3444 [==============================] - 75s 22ms/step - loss: 0.5807 - accuracy: 0.6447 - val_loss: 0.7798 - val_accuracy: 0.5317
Epoch 7/10
3444/3444 [==============================] - 75s 22ms/step - loss: 0.5696 - accuracy: 0.6534 - val_loss: 0.8031 -

In [7]:
X_test = pd.read_csv("/content/drive/MyDrive/nlp/test_content_all.csv").dropna()
X_test = X_test[X_test["form_type"] != "4"]
tokenized_texts_test = tokenizer.texts_to_sequences(X_test['content'])
X_test_input = sequence.pad_sequences(tokenized_texts_test, maxlen=500)

In [8]:
y_pred = model.predict(X_test_input)
X_test["preds"] = y_pred

In [9]:
import numpy as np
X_test["preds_prob"] = X_test["preds"]
X_test["preds"] = np.where(X_test["preds_prob"] >= 0.5, 1, 0)
X_test

,file_number,ticker,form_type,label,content,preds,preds_prob
7,274787,ARE,8-K,0.0,shares shares false false false false false pa...,0,0.062376
8,274788,ED,3,0.0,undersigned constitutes appoints signing power...,1,0.518297
9,274789,ED,3,0.0,undersigned constitutes appoints signing power...,1,0.518297
17,274797,CNC,8-K,0.0,false false false false false flag true conten...,0,0.056435
18,274798,CNC,8-K,0.0,false false false false false flag true conten...,0,0.056435
...,...,...,...,...,...,...,...
61546,527639,FRLG,8-K,0.0,corporation organized existing accordance prov...,0,0.051888
61547,527640,FRLG,FWP,0.0,pursuant dated notes bear paid notes stated ma...,0,0.206636
61548,527641,FRLG,FWP,0.0,pursuant dated securities unsecured notes issu...,0,0.427927
61549,527642,FRLG,FWP,0.0,pursuant dated notes bear paid notes stated ma...,0,0.469839


In [10]:
X_test.to_csv("/content/drive/MyDrive/nlp/rnn.csv")